In [4]:
import random
import json
import pickle
import numpy as np
import tensorflow as tf
import io
import pandas as pd
from google.colab import files

uploaded = files.upload()

import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
intents = pd.read_json(io.StringIO(uploaded.get('intents.json').decode('utf-8')))

words = []
classes = []
documents = []
ignoreLetters = ['?', '!', '.', ',']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        wordList = nltk.word_tokenize(pattern)
        words.extend(wordList)
        documents.append((wordList, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(word) for word in words if word not in ignoreLetters]
words = sorted(set(words))

classes = sorted(set(classes))

pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

training = []
outputEmpty = [0] * len(classes)

for document in documents:
    bag = []
    wordPatterns = document[0]
    wordPatterns = [lemmatizer.lemmatize(word.lower()) for word in wordPatterns]
    for word in words:
        bag.append(1) if word in wordPatterns else bag.append(0)

    outputRow = list(outputEmpty)
    outputRow[classes.index(document[1])] = 1
    training.append(bag + outputRow)

random.shuffle(training)
training = np.array(training)

trainX = training[:, :len(words)]
trainY = training[:, len(words):]

model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(128, input_shape=(len(trainX[0]),), activation = 'relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(64, activation = 'relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(len(trainY[0]), activation='softmax'))

sgd = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

model.fit(trainX, trainY, epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5')
print('Done')

Saving intents.json to intents (1).json


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Epoch 1/200
47/47 [==============================] - 1s 2ms/step - loss: 4.3846 - accuracy: 0.0259
Epoch 2/200
47/47 [==============================] - 0s 2ms/step - loss: 4.2982 - accuracy: 0.0560
Epoch 3/200
47/47 [==============================] - 0s 2ms/step - loss: 4.2036 - accuracy: 0.0603
Epoch 4/200
47/47 [==============================] - 0s 2ms/step - loss: 4.1377 - accuracy: 0.0733
Epoch 5/200
47/47 [==============================] - 0s 2ms/step - loss: 4.0067 - accuracy: 0.0948
Epoch 6/200
47/47 [==============================] - 0s 2ms/step - loss: 3.8640 - accuracy: 0.1293
Epoch 7/200
47/47 [==============================] - 0s 2ms/step - loss: 3.7398 - accuracy: 0.0905
Epoch 8/200
47/47 [==============================] - 0s 2ms/step - loss: 3.5694 - accuracy: 0.1638
Epoch 9/200
47/47 [==============================] - 0s 2ms/step - loss: 3.4670 - accuracy: 0.1724
Epoch 10/200
47/47 [==============================] - 0s 2ms/step - loss: 3.3070 - accuracy: 0.1897
Epoch 11/

In [ ]:
import random
import json
import pickle
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
import tensorflow
from keras.models import load_model
nltk.download('punkt')
lematizer = WordNetLemmatizer()
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))
model = load_model('chatbot_model.h5')

def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lematizer.lemmatize(word) for word in sentence_words]
    return sentence_words

def bag_of_words(sentence):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)
    for w in sentence_words:
        for i, word in enumerate(words):
            if word == w:
                bag[i]=1
    return np.array(bag)

def predict_class(sentence):
    bow = bag_of_words(sentence)
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    results.sort(key=lambda x:x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({'intent':classes[r[0]],'probability':str(r[1])})
    return return_list

def get_response(intents_list, intents_json):
    tag = intents_list[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
    return result
print("Bot is running Hell yeah!")
while True:
    message = input("")
    ints = predict_class(message)
    res = get_response(ints, intents)
    print(res)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Bot is running Hell yeah!
Hey
1/1 [==============================] - 0s 68ms/step
Hi there. How are you feeling today?
Mentally stressed
1/1 [==============================] - 0s 35ms/step
Give yourself a break. Go easy on yourself.
